# Re-challenge CIFAR100

In P1 chapter7, we use pretrained `wide_resnet50_2` to fine-tuning on CIFAR100, and finally got around 79% accuracy. In P3, we learned many useful techniques for model training and in this chapter, we try to use them make our results on CIFAR100 better without changing model.

## Load Model

In [1]:
import torchvision.models as models
import torch.nn as nn

def init_model():
    model = models.wide_resnet50_2(pretrained=True)
    model.conv1 = nn.Conv2d(model.conv1.in_channels,
                            model.conv1.out_channels,
                            3, 1, 1)
    model.maxpool = nn.Identity() # nn.Conv2d(64, 64, 1, 1, 1)
    model.fc = nn.Linear(model.fc.in_features, 100)
    return model

## Label Smoothing

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class LabelSmoothCEloss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,  pred,  label,  smoothing=0.1):
        pred = F.softmax(pred,  dim=1)
        one_hot_label = F.one_hot(label, pred.size(1)).float()
        smoothed_one_hot_label = (1.0 - smoothing)  *  one_hot_label + smoothing / pred.size(1)
        loss = (-torch.log(pred))  *  smoothed_one_hot_label
        loss = loss.sum(axis=1,  keepdim=False)
        loss = loss.mean()
        return loss

## Cutmix

In [3]:
import torch
import numpy as np


def cutmix(data, targets, alpha=1.0):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)

    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    new_data = data.clone()
    new_data[:, :, bbx1:bbx2, bby1:bby2] = shuffled_data[:, :, bbx1:bbx2, bby1:bby2]

    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size(-1) * data.size(-2)))

    targets = (targets, shuffled_targets, lam)
    return new_data, targets


def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2


def cutmix_loss(criterion, outputs, targets):
    targets, shuffled_targets, lam = targets
    loss = lam * criterion(outputs, targets) + (1 - lam) * criterion(outputs, shuffled_targets)
    return loss

## Train Function

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import random
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import LambdaLR, CosineAnnealingLR
from tqdm import tqdm
import matplotlib.pyplot as plt
import datetime


class cifar100_trainer_v2:
    def __init__(self, model, batch_size, epoch, train_transforms, test_transforms, seed=1234):
        self.model = model
        self.epoch = epoch

        self.train_losses = []
        self.val_losses = []
        self.val_accuracies = []

        """load model, loss_fn and optimizer"""
        if torch.cuda.is_available():
            self.model = self.model.cuda()
        self.scaler = GradScaler()

        """download and allocate data"""
        ROOT = '.data'
        train_valid_data = torchvision.datasets.CIFAR100(root=ROOT, train=True, download=True, transform=train_transforms)
        test_data = torchvision.datasets.CIFAR100(root=ROOT, train=False, download=False, transform=test_transforms)

        VALID_RATE = 0.2
        n_train = int(len(train_valid_data) * (1 - VALID_RATE))
        n_valid = len(train_valid_data) - n_train
        train_data, valid_data = torch.utils.data.random_split(train_valid_data, [n_train, n_valid])

        """load data"""
        self.train_dataloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
        self.valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size)
        self.test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

        """set the seed"""
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

    def train(self):
        criterion = LabelSmoothCEloss()
        if torch.cuda.is_available():
            criterion = criterion.cuda()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        def lr_lambda(epoch):
            warmup_epochs = 10
            if epoch < warmup_epochs:
                return (epoch + 1) / float(warmup_epochs)
            else:
                return 0.1 ** ((epoch - warmup_epochs) / 10)

        scheduler_warmup = LambdaLR(optimizer, lr_lambda, last_epoch=-1)
        T_max = 10
        eta_min = 0

        for epoch in range(self.epoch):
            print('=' * 135)
            print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            print(f"Epoch {epoch + 1} / {self.epoch}")
            self.model.train()
            train_bar = tqdm(self.train_dataloader, desc="[Train]", ncols=90)
            train_losses = []

            if epoch < 10:
                scheduler_warmup.step()
            else:
                scheduler_cosine = CosineAnnealingLR(optimizer, T_max=T_max, eta_min=eta_min, last_epoch=epoch-1)
                scheduler_cosine.step()

            for inputs, targets in train_bar:
                if torch.cuda.is_available():
                    inputs, targets = inputs.cuda(), targets.cuda()
                inputs, targets = cutmix(inputs, targets)
                optimizer.zero_grad()
                with autocast(enabled=torch.cuda.is_available()):
                    outputs = self.model(inputs)
                    loss = cutmix_loss(criterion, outputs, targets)

                self.scaler.scale(loss).backward()
                self.scaler.step(optimizer)
                self.scaler.update()
                train_losses.append(loss.item())
                train_bar.set_postfix(train_loss=sum(train_losses) / len(train_bar))
            train_bar.close()
            self.train_losses.append(sum(train_losses) / len(train_bar))

            self.model.eval()
            valid_loss = 0
            valid_accuracy = 0
            valid_samples = 0
            valid_bar = tqdm(self.valid_dataloader, desc="[Valid]", ncols=100)
            with torch.no_grad():
                for inputs, targets in valid_bar:
                    if torch.cuda.is_available():
                        inputs, targets = inputs.cuda(), targets.cuda()
                    outputs = self.model(inputs)
                    loss = criterion(outputs, targets)

                    valid_loss += loss.item()
                    pred = outputs.argmax(1)
                    correct = (pred == targets).sum().item()
                    valid_accuracy += correct
                    valid_samples += targets.size(0)
                    val_loss = valid_loss / valid_samples
                    val_acc = valid_accuracy / valid_samples
                    valid_bar.set_postfix(val_loss=val_loss, val_acc=val_acc * 100)

            valid_bar.close()
            self.val_losses.append(val_loss)
            self.val_accuracies.append(val_acc)

        self.plot_training()

    def plot_training(self):
        plt.figure(figsize=(10, 5))
        plt.plot(range(1, self.epoch + 1), self.train_losses, label='Train Loss')
        plt.plot(range(1, self.epoch + 1), self.val_losses, label='Val Loss')
        plt.plot(range(1, self.epoch + 1), self.val_accuracies, label='Val Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Metrics')
        plt.title('Training Progress')
        plt.legend()
        plt.show()

    def test(self):
        criterion = LabelSmoothCEloss()
        if torch.cuda.is_available():
            criterion = criterion.cuda()
        self.model.eval()
        total_test_loss = 0
        total_test_accuracy = 0
        num_samples = 0
        test_bar = tqdm(self.test_dataloader, desc="Testing [Test]")
        with torch.no_grad():
            for inputs, targets in test_bar:
                if torch.cuda.is_available():
                    inputs = inputs.cuda()
                    targets = targets.cuda()
                outputs = self.model(inputs)
                loss = criterion(outputs, targets)

                total_test_loss += loss.item()
                accuracy = (outputs.argmax(1) == targets).sum().item()
                total_test_accuracy += accuracy
                num_samples += len(inputs)
                test_bar.set_postfix(test_loss=total_test_loss / num_samples,
                                     test_acc=total_test_accuracy / num_samples * 100)
                test_bar.update()
        test_bar.close()

## Start Training

In [ ]:
from torchvision.transforms import autoaugment, transforms


train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(0.08, 1.0), ratio=(3. / 4., 4. / 3.)),
    transforms.RandomHorizontalFlip(),
    autoaugment.AutoAugment(policy=autoaugment.AutoAugmentPolicy('cifar10')),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

model = init_model()
trainer = cifar100_trainer_v2(model=model, batch_size=256, epoch=200, train_transforms=train_transforms, test_transforms=test_transforms)
trainer.train()
trainer.test()

Files already downloaded and verified
2024-06-01 15:10:10
Epoch 1 / 200


[Valid]: 100%|███████████████████████| 40/40 [00:28<00:00,  1.38it/s, val_acc=12.6, val_loss=0.0158]


2024-06-01 15:12:47
Epoch 2 / 200


[Valid]: 100%|███████████████████████| 40/40 [00:28<00:00,  1.42it/s, val_acc=19.2, val_loss=0.0146]


2024-06-01 15:15:22
Epoch 3 / 200


...

[Valid]: 100%|██████████████████████| 40/40 [00:27<00:00,  1.43it/s, val_acc=52.5, val_loss=0.00958]


2024-06-01 18:33:32
Epoch 80 / 200


[Valid]: 100%|██████████████████████| 40/40 [00:27<00:00,  1.45it/s, val_acc=53.8, val_loss=0.00933]


2024-06-01 18:36:06
Epoch 81 / 200


[Train]:  16%|███▉                     | 25/157 [00:21<01:48,  1.22it/s, train_loss=0.493]